In [ ]:
! wget -nc https://www.dropbox.com/s/i1svta8esb3f36j/data.zip?dl=1 -O data.zip
! unzip -n data.zip
! pip install spectrum-utils[iplot] tqdm

In [13]:
import csv
import re
import pickle
import spectrum_utils.spectrum as sus
from tqdm import tqdm
import numpy as np
import pandas as pd

MASS_SHIFT = ["+0.984", "+42.011", "+15.995", "-17.027", "+43.006", "+57.021"]
usi_prefix = "mzspec:PXD010154:"
fragment_tol_mass, fragment_tol_mode = 0.05, "Da"
data_type = "train"
skip_rows = 15001
nrows = 15000

with open(f"./data/{data_type}/spectrum.pkl", "rb") as f:
    data = pickle.load(f)

# Open the TSV file
df = pd.read_csv(f"data/{data_type}/raw/{data_type}.tsv", delimiter="\t", skiprows=range(1, skip_rows), nrows=15000)
scan_number = 0
annotations = {}
# Iterate through each row
for _, row in tqdm(df.iterrows()):
    if scan_number != row["ScanNum"]:
        scan_number = int(row["ScanNum"])
        original_filepath = row["OriginalFilepath"]
        # Extract the filename from the original filepath
        filename = original_filepath.split("/")[-1]
        # print(filename)
        usi = f"{usi_prefix}{filename}:scan:{scan_number}"
        precursor_mz = float(row["Precursor"])
        precursor_charge = int(row["Charge"])
        mz = data[filename][scan_number]["mz_arr"]
        intensity = data[filename][scan_number]["intensity_arr"]
        # Create the spectrum object
        spectrum = sus.MsmsSpectrum(usi, precursor_mz, precursor_charge, mz, intensity)
        spectrum = (
            spectrum.set_mz_range(min_mz=0, max_mz=2000)
            .remove_precursor_peak(fragment_tol_mass, fragment_tol_mode)
            .scale_intensity("root", max_intensity=1.0)
        )

    peptide = row["Peptide"]
    match = re.search(r"\.(.+)\.", peptide)
    peptide = match.group(1)
    for mass_shift in MASS_SHIFT:
        if mass_shift in peptide:
            peptide = peptide.replace(mass_shift, "[" + mass_shift + "]")
    start_idx = peptide.find("[-17")
    end_idx = peptide.find("]Q")
    if start_idx != -1 and end_idx != -1 and start_idx < end_idx:
        peptide = (
            peptide[:start_idx]
            + "Q"
            + peptide[start_idx : end_idx + 1]
            + peptide[end_idx + 2 :]
        )
    if peptide.startswith("["):
        peptide = peptide.replace("]", "]-", 1)

    theospec = spectrum.annotate_proforma(
        peptide,
        fragment_tol_mass,
        fragment_tol_mode,
        ion_types="abcxyz",
        max_ion_charge=precursor_charge,
        neutral_losses={"NH3": -17.026549, "H20": -18.010565},
    )

    if filename not in annotations:
        annotations[filename] = {}
    if scan_number not in annotations[filename]:
        annotations[filename][scan_number] = {}
    if "usi" not in annotations[filename][scan_number]:
        annotations[filename][scan_number]["usi"] = usi
    if "charge" not in annotations[filename][scan_number]:
        annotations[filename][scan_number]["charge"] = precursor_charge
    if "mz_arr" not in annotations[filename][scan_number]:
        annotations[filename][scan_number]["mz_arr"] = np.float32(spectrum.mz)
    if "intensity_arr" not in annotations[filename][scan_number]:
        annotations[filename][scan_number]["intensity_arr"] = np.float32(spectrum.intensity)

    if "annotations_arr" not in annotations[filename][scan_number]:
        annotations[filename][scan_number]["annotations_arr"] = []
    annotations[filename][scan_number]["annotations_arr"] += [
        [str(el) for el in theospec.annotation]
    ]
    if "peptides" not in annotations[filename][scan_number]:
        annotations[filename][scan_number]["peptides"] = []
    annotations[filename][scan_number]["peptides"] += [row["Peptide"]]
    if "proteins" not in annotations[filename][scan_number]:
        annotations[filename][scan_number]["proteins"] = []
    annotations[filename][scan_number]["proteins"] += [row["Protein"]]
    if "spec_evalue" not in annotations[filename][scan_number]:
        annotations[filename][scan_number]["spec_evalue"] = []
    annotations[filename][scan_number]["spec_evalue"] += [row["SpecEValue"]]
    break

0it [00:00, ?it/s]


In [14]:
annotations

{'01088_A05_P010740_S00_N33_R1.mzML': {9590: {'usi': 'mzspec:PXD010154:01088_A05_P010740_S00_N33_R1.mzML:scan:9590',
   'charge': 3,
   'mz_arr': array([ 107.04947 ,  110.07153 ,  111.074745,  112.08676 ,  116.07104 ,
           119.04923 ,  120.08148 ,  129.06602 ,  129.10237 ,  130.04985 ,
           130.08632 ,  136.07582 ,  137.07947 ,  138.06621 ,  147.04376 ,
           147.07666 ,  147.11258 ,  149.02379 ,  155.08147 ,  157.09734 ,
           157.10893 ,  157.13339 ,  158.09277 ,  159.0916  ,  165.05481 ,
           173.09229 ,  175.11911 ,  176.1225  ,  181.10828 ,  182.0813  ,
           183.07632 ,  184.05843 ,  185.12898 ,  185.16452 ,  186.08646 ,
           187.07181 ,  193.09727 ,  200.12155 ,  201.08722 ,  202.08412 ,
           203.42903 ,  207.01749 ,  209.10292 ,  209.12619 ,  209.62743 ,
           210.68677 ,  211.10707 ,  213.1597  ,  221.09157 ,  222.01875 ,
           230.0779  ,  236.0671  ,  237.13431 ,  243.13411 ,  244.64485 ,
           245.07582 ,  246.9385

In [8]:
import pickle

data_type = "test"
annotations = {}
for i in range(1, 4):
    with open(f"./data/{data_type}/annotations_{i}.pkl", "rb") as f:
        dct = pickle.load(f)
        for key in dct.keys():
            if key not in annotations:
                annotations[key] = {}
            annotations[key].update(dct[key])

In [9]:
with open(f"./data/{data_type}/annotations.pkl", "wb") as f:
    pickle.dump(annotations, f)